In [0]:
dbutils.widgets.removeAll()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from decimal import Decimal

In [0]:
dbutils.widgets.text("catalogo", "catalog_xbautisv")
dbutils.widgets.text("esquema_source", "silver")
dbutils.widgets.text("esquema_sink", "golden")

In [0]:
catalogo = dbutils.widgets.get("catalogo")
esquema_source = dbutils.widgets.get("esquema_source")
esquema_sink = dbutils.widgets.get("esquema_sink")

## Llamado de la tabla pre-procesada en Silver

In [0]:
df_orden_compra_transformed = spark.table(f"{catalogo}.{esquema_source}.ordenes_compra")

## Realizo cálculo para mostrar información de
## monto acumulado por pedido y cliente sobre la tabla silver

In [0]:
window_spec = (
    Window
    .partitionBy("id_pedido","id_cliente")          # PARTITION BY
    .orderBy("id_item")               # ORDER BY
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

In [0]:
df_acumulado = df_orden_compra_transformed.withColumn("monto_total",F.sum("subtotal").over(window_spec))

In [0]:
window_item = (
    Window
    .partitionBy("id_pedido")
    .orderBy(F.col("id_item").desc())
)

##Solo mostrará un registro con el total acumulado por cliente y pedido

In [0]:
df_ultimo_item = (
    df_acumulado
    .withColumn("rn", F.row_number().over(window_item))
    .filter(F.col("rn") == 1)
    .orderBy("id_item")
)

In [0]:
orden_sector_df = df_ultimo_item.select(col("id_pedido"),
                                                    col("id_cliente"),
                                                    col("nombre_cliente"),
                                                    col("segmento_cliente"),
                                                    col("fecha_pedido"),
                                                    col("fecha_envio"),
                                                    col("metodo_envio"),
                                                    col("ciudad"),
                                                    col("provincia"),
                                                    col("pais"),
                                                    col("region"),
                                                    col("nombre_producto"),
                                                    col("categoria"),
                                                    col("subcategoria"),
                                                    col("pedido_devuelto"),
                                                    col("gerente_regional"),
                                                    col("monto_total"))

In [0]:
spark.sql(f"DROP TABLE IF EXISTS {catalogo}.{esquema_sink}.ordenes_compra")

In [0]:
orden_sector_df.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_sink}.orden_compra_sectorizada")